In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
import random

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True


In [4]:
y = train['label']
x = train.drop('label', axis=1)

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x = scaler.fit_transform(x)
test = scaler.transform(test)

In [6]:
label_cnt = len(y.unique())

In [7]:
W = torch.zeros((x.shape[1], label_cnt), requires_grad=True)
b = torch.zeros(label_cnt, requires_grad=True)

In [8]:
x_tensor = torch.FloatTensor(x)
y_tensor = torch.LongTensor(y)
test_tensor = torch.FloatTensor(test)

In [9]:
optimizer = torch.optim.SGD([W, b], lr=0.1)

epochs = 1000

for epoch in range(epochs):
    hypothesis = torch.nn.functional.softmax(torch.mm(x_tensor, W) + b, dim=1)
    cost = torch.nn.functional.cross_entropy(torch.mm(x_tensor, W) + b, y_tensor)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(f'Epoch: {epoch:4d}/{epochs} Cost: {cost.item():.6f}')

Epoch:    0/1000 Cost: 2.302585
Epoch:  100/1000 Cost: 0.345892
Epoch:  200/1000 Cost: 0.310402


KeyboardInterrupt: 

In [ ]:
hypothesis = torch.nn.functional.softmax(test_tensor.matmul(W) + b, dim=1) # or .mm or @
predict = torch.argmax(hypothesis, dim=1)

In [ ]:
submit = pd.read_csv('sample_submit.csv')
submit.label = predict
submit.to_csv('submit.csv', index=False)